In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
import numpy as np

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10
learning_rate = 0.0001                        # NEW 0.001


X_train=mnist.train.images.reshape((-1, n_steps, n_inputs))
# X_train=mnist.train.images
Y_train=mnist.train.labels

X_test=mnist.test.images.reshape((-1, n_steps, n_inputs))
# X_test=mnist.test.images
Y_test=mnist.test.labels

X_val=mnist.validation.images.reshape((-1, n_steps, n_inputs))
# X_val=mnist.validation.images
Y_val=mnist.validation.labels

In [4]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

# X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))
basic_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, state_is_tuple=False)

outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)


logits_0_0 = tf.abs(fully_connected(states, n_steps*1, activation_fn=None))

logits_0 = tf.reshape(logits_0_0, shape=[-1, n_steps, 1])

basic_cell_1 = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

outputs_1, states_1 = tf.nn.dynamic_rnn(basic_cell_1, logits_0, dtype=tf.float32)


logits = tf.abs(fully_connected(states_1, n_steps*n_inputs, activation_fn=tf.nn.relu))

# logits = fully_connected(states_1, n_outputs, activation_fn=None)


# xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

X1=tf.reshape(X, shape=[-1, n_steps*n_inputs])
logits_1=tf.reshape(logits, shape=[-1, n_steps, n_inputs])

xentropy=tf.reduce_sum(tf.abs(tf.subtract(X1,logits))) # NEW

                 

loss = tf.reduce_mean(xentropy)


optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
# correct = tf.nn.in_top_k(logits, y, 1)
# accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [5]:
init = tf.global_variables_initializer()

In [ ]:
n_epochs = 7
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#         print(sess.run(logits_0,feed_dict={X: X_batch}))    # NEW
#         acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
#         acc_test = accuracy.eval(feed_dict={X: X_test, y: Y_test})
        loss_ = loss.eval(feed_dict={X: X_test, y: Y_test})            # NEW
#         print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test,"Loss:",loss_)
        print("Loss:",loss_)
    a=sess.run(logits_1,feed_dict={X: X_test})

Loss: 757104.8
Loss: 616088.4
Loss: 566589.1


In [ ]:
import matplotlib.pyplot as plt

pixels = np.array(a, dtype='uint8')
index =150
plt.imshow(a[index,:,:])

In [ ]:
index = 150
plt.imshow(X_test[index,:,:])